In [43]:
import numpy as np
import pandas as pd
import json
import re
import MeCab
from collections import Counter

In [138]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [46]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]
mecab_param = MeCab.Tagger("-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

In [123]:
NITE_existence_category = pd.read_csv("../data/NITE_existence_list.csv")
NITE_existence_category = NITE_existence_category.loc[NITE_existence_category.label == 1].reset_index(drop=True)
NITE_existence_category.category = NITE_existence_category.category.apply(lambda x: re.sub(r'の', '', x) if re.match(r'.+の化合物', x) else x)
NITE_existence_category.head()

,category,label
0,アルコール,1
1,アミン,1
2,フェノールエーテル,1
3,ケトン,1
4,無機化合物,1


In [4]:
with open("../data/compound_train.json") as f:
    train = json.load(f)['entry']

In [131]:
train_type_dict = dict([[str(entry['WikipediaID']), entry['Attributes']['種類']] for entry in train])
train_type_list = list(set(flatten(train_type_dict.values())))

In [ ]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

In [251]:
train_wiki_doc_df = \
pd.DataFrame({
    "_id": [entry['index']['_id'] for entry in wiki_data]
    , "document": [entry['opening_text'] if ('opening_text' in entry.keys() and entry['opening_text'] != None) else entry['text'] for entry in wiki_data]
    , "category": [entry['category'] for entry in wiki_data]
}).pipe(
    lambda x: x.loc[x._id.isin(train_type_dict.keys())]
).reset_index(drop=True)

In [295]:
def get_noun_list(doc):
    mecab_param = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    mecab_param.parse("")
    node = mecab_param.parseToNode(doc)
    
    noun_list = []
    noun = []
    while node:
        if len(node.surface) == 0:
            node = node.next
            continue

        hinshi = node.feature.split(',')
        if hinshi[0] == '名詞':
            noun.append(node.surface)
        elif len(noun) > 0:            
            noun_list.append(''.join(noun))
            noun = []
        
        node = node.next
    
    if len(noun) > 0:
        noun_list.append(''.join(noun))
    

    return noun_list

In [401]:
# NITEに存在するカテゴリ名と一致する単語を抽出

type_list = NITE_existence_category.category.tolist()

type_using_NITE_df = \
train_wiki_doc_df.set_index('_id').apply(
    lambda x: pd.Series(type_list, index=type_list).isin(get_noun_list(x.document))
    , axis=1
)

In [398]:
# 各ページに存在するカテゴリ名と一致する単語を抽出
type_using_categories_df = \
train_wiki_doc_df.set_index('_id').apply(
    lambda x: pd.Series(x.category, index=x.category).isin(get_noun_list(x.document))
    , axis=1
).fillna(False)

In [406]:
type_using_NITE_list = \
type_using_NITE_df.apply(
    lambda x: x.index.values[x.values].tolist()
    , axis=1
).tolist()

type_using_categories_list = \
type_using_categories_df.apply(
    lambda x: x.index.values[x.values].tolist()
    , axis=1
).tolist()

extracted_type_list = [list(set(type1 + type2)) for type1, type2 in zip(type_using_NITE_list, type_using_categories_list)]

In [407]:
extracted_type_dict = \
dict(
    zip(
        type_using_NITE_df.index
        , extracted_type_list
    )
)

## Validation

In [408]:
result_precision = []
result_recall = []
for _id, types in extracted_type_dict.items():
    result_precision += [(_type in train_type_dict[_id]) for _type in types]
    result_recall += [(train_type in types) for train_type in train_type_dict[_id]]

In [409]:
precision = sum(result_precision) / len(result_precision)
recall = sum(result_recall) / len(result_recall)
f1 = 2 * precision * recall / (precision + recall)

print(f"Precision:{precision}")
print(f"Recall:{recall}")
print(f"f1:{f1}")

Precision:0.4307511737089202
Recall:0.4860927152317881
f1:0.45675171126322345


In [411]:
extracted_type_dict

{'2662912': ['炭化水素'],
 '652294': ['無機化合物'],
 '1533538': ['爆薬', 'アニリン', '水素'],
 '184525': ['ポリオール', 'アルコール', 'グリコール'],
 '497499': ['ピロリドン', '有機化合物'],
 '1273610': ['塩'],
 '1302688': [],
 '2639623': ['アントラサイクリン'],
 '22229': ['アルカン'],
 '847474': [],
 '616186': [],
 '3457795': [],
 '2153576': [],
 '2306663': [],
 '610199': [],
 '1259267': ['塩', '有機化合物', 'アニオン', 'エステル'],
 '899679': [],
 '1565134': ['塩', '亜塩素酸塩'],
 '804585': ['有機化合物', 'アミン'],
 '487740': ['無機化合物', '殺虫剤', '除草剤', '塩化物', '毒物', '可塑剤'],
 '841166': ['酸'],
 '770695': ['溶媒', 'アミド', '有機化合物', 'エーテル', 'アルコール', 'ケトン'],
 '1302712': ['アルコール', '有機化合物', 'テトラヒドロピラン'],
 '86438': ['脂肪酸', 'エーテル', 'カルボン酸'],
 '2370416': ['雷酸塩'],
 '1064260': ['有機アルミニウム化合物', '有機化合物'],
 '626464': [],
 '1448447': ['溶媒', 'エーテル', '有機化合物'],
 '1566859': ['有機化合物'],
 '3083097': [],
 '73370': ['アルカン'],
 '804622': ['有機化合物'],
 '3377784': [],
 '236147': ['有機金属化合物', '塩基'],
 '32765': ['カルボン酸'],
 '1793404': ['臭化物'],
 '1268469': ['芳香族化合物', '有機化合物'],
 '1866600': ['有機化合物'],
 '56091': 

In [416]:
Counter(flatten(extracted_type_dict.values())).most_common()

[('有機化合物', 92),
 ('無機化合物', 53),
 ('アルコール', 19),
 ('塩', 19),
 ('水素', 18),
 ('酸化物', 17),
 ('酸化剤', 14),
 ('アミノ酸', 13),
 ('医薬品', 13),
 ('塩化物', 12),
 ('酸', 12),
 ('エーテル', 12),
 ('芳香族化合物', 12),
 ('タンパク質', 12),
 ('炭酸塩', 11),
 ('塩基', 10),
 ('水酸化物', 10),
 ('酢酸', 10),
 ('触媒', 10),
 ('アルカロイド', 9),
 ('炭化水素', 8),
 ('アミン', 8),
 ('カルボン酸', 8),
 ('食品添加物', 8),
 ('アニオン', 7),
 ('エステル', 7),
 ('殺虫剤', 7),
 ('溶媒', 7),
 ('劇物', 7),
 ('毒物', 6),
 ('ケトン', 6),
 ('臭化物', 6),
 ('アンモニウム塩', 6),
 ('複素環式化合物', 6),
 ('アルケン', 6),
 ('アルカン', 5),
 ('硫酸塩', 5),
 ('ヨウ化物', 5),
 ('有機リン化合物', 5),
 ('過酸化物', 5),
 ('テルペノイド', 5),
 ('爆薬', 4),
 ('脂肪酸', 4),
 ('安息香酸', 4),
 ('ニトリル', 4),
 ('農薬', 4),
 ('色素', 4),
 ('糖類', 4),
 ('酸無水物', 4),
 ('精神刺激薬', 4),
 ('尿素', 4),
 ('ピリジン', 4),
 ('ラクトン', 4),
 ('硝酸塩', 4),
 ('有機硫黄化合物', 4),
 ('有機金属化合物', 3),
 ('インドール', 3),
 ('配糖体', 3),
 ('香料', 3),
 ('オキソ酸', 3),
 ('芳香族炭化水素', 3),
 ('染料', 3),
 ('神経毒', 3),
 ('抗不安薬', 3),
 ('シアン化物', 3),
 ('アルデヒド', 3),
 ('チオール', 3),
 ('モノマー', 3),
 ('セスキテルペン', 3),
 ('ジカルボン酸', 3),
 ('官能基', 3

In [293]:
train_type_dict

{'1052083': ['化合物'],
 '1385415': ['樹脂酸', '有機酸'],
 '1023103': ['モノテルペンアルコール'],
 '1024773': ['アセン', '多環芳香族炭化水素'],
 '1027018': ['色素', 'カロテノイド系'],
 '10166': ['無機化合物'],
 '1041689': ['芳香族炭化水素', '有機化合物'],
 '1050477': [],
 '1059604': ['ヒドロキシ酸'],
 '1064260': ['有機化合物'],
 '1067475': ['蛍光色素'],
 '1073480': [],
 '1079879': [],
 '1083386': ['マクロライド系抗生物質'],
 '1095173': ['塩'],
 '11006': ['炭化水素'],
 '1105111': ['チアミン誘導体'],
 '1198019': ['テルペノイド'],
 '1139287': [],
 '1140453': [],
 '1140516': [],
 '1145214': ['コリンエステル類'],
 '1148105': [],
 '1151355': ['ヌクレオシド'],
 '1156072': [],
 '1162928': ['芳香族化合物', '誘導体'],
 '1164069': ['化合物の混合物', 'セスキテルペン'],
 '1170449': ['中間生成物'],
 '1171034': [],
 '1180712': [],
 '1185967': ['有機化合物'],
 '1124544': ['化合物'],
 '1203950': ['NSAID', '非ステロイド性抗炎症薬'],
 '1205115': ['抗コリン薬'],
 '1210379': ['精神安定剤', 'トランキライザー'],
 '1215871': ['交感神経作用アミン'],
 '1235986': ['アミノ酸'],
 '1237720': ['単糖'],
 '1237927': ['第二級アミン'],
 '1238745': ['不定比化合物', '鉄の空格子を多量に含んだ結晶'],
 '12437': ['第一級アルコール', 'アルコール類'],
 '12444

In [415]:
Counter(flatten(train_type_dict.values())).most_common(20)

[('有機化合物', 86),
 ('無機化合物', 48),
 ('化合物', 46),
 ('酸化物', 13),
 ('誘導体', 12),
 ('塩', 9),
 ('アルカロイド', 9),
 ('フッ化物', 8),
 ('塩化物', 8),
 ('芳香族化合物', 7),
 ('アミノ酸', 7),
 ('炭酸塩', 7),
 ('カルボン酸', 7),
 ('水酸化物', 7),
 ('複素環式化合物', 6),
 ('炭化水素', 5),
 ('有機リン化合物', 5),
 ('オキソ酸', 5),
 ('臭化物', 5),
 ('有機硫黄化合物', 5)]